% This LaTeX was auto-generated from MATLAB code.
% To make changes, update the MATLAB code and export to LaTeX again.

\documentclass{article}

\usepackage[utf8]{inputenc}
\usepackage[T1]{fontenc}
\usepackage{lmodern}
\usepackage{graphicx}
\usepackage{color}
\usepackage{hyperref}
\usepackage{amsmath}
\usepackage{amsfonts}
\usepackage{epstopdf}
\usepackage[table]{xcolor}
\usepackage{matlab}

\sloppy
\epstopdfsetup{outdir=./}
\graphicspath{ {./ELC04_Sistemas_Segundo_Orden_images/} }

\matlabhastoc

\begin{document}

\begin{matlabcode}
clear all
\end{matlabcode}

\label{H_44243527}
\matlabheading{Repaso General de Teoría de Control - Ezequiel Leonardo Castaño © - 2021}

\matlabtableofcontents{Table of Contents}
\label{H_146FB83C}
\matlabheading{Sistemas de Segundo Orden}

\begin{par}
\begin{flushleft}
Dado un Porcentaje de Overshoot (PO)
\end{flushleft}
\end{par}

\begin{matlabcode}
PO = 15
\end{matlabcode}
\begin{matlaboutput}
PO = 15
\end{matlaboutput}

\label{H_9977F9A8}
\matlabheadingtwo{Calculando ζ}

\begin{par}
\begin{flushleft}
Despejamos el factor de amortiguanmiento relativo de la fórmula de overshoot
\end{flushleft}
\end{par}

\begin{par}
\begin{flushleft}
Se definen las dos variables
\end{flushleft}
\end{par}

\begin{matlabcode}
overshoot = sym("PO", 'positive'); 
zeta = sym("zeta", 'positive');
\end{matlabcode}

\begin{par}
\begin{flushleft}
Se escribe la fómula de overshoot relativo - Bolton p100)
\end{flushleft}
\end{par}

\begin{matlabcode}
overshoot_equation = overshoot == exp((-zeta * pi) / sqrt(1 - zeta^2));
\end{matlabcode}

\begin{par}
\begin{flushleft}
Se despeja y se muestra la solución
\end{flushleft}
\end{par}

\begin{matlabcode}
zeta_equation = simplify(solve(overshoot_equation, zeta, 'Real', true));
\end{matlabcode}
\begin{matlaboutput}
Warning: Solutions are only valid under certain conditions. To include parameters and conditions in the solution, specify the 'ReturnConditions' value as 'true'.
\end{matlaboutput}
\begin{matlabcode}
pretty(zeta_equation)
\end{matlabcode}
\begin{matlaboutput}
      |log(PO)|
--------------------
       2          2
sqrt(pi  + log(PO) )
\end{matlaboutput}

\begin{par}
\begin{flushleft}
Se escribe la fórmula en código y se calcula ζ
\end{flushleft}
\end{par}

\begin{matlabcode}
zeta = abs(log(PO/100)) / sqrt(pi^2 + log(PO/100)^2)
\end{matlabcode}
\begin{matlaboutput}
zeta = 0.5169
\end{matlaboutput}


\label{H_811BB9E2}
\matlabheadingtwo{Definiendo sistema de Segundo Orden}

\begin{par}
\begin{flushleft}
Se asume un valor arbitrario para ω
\end{flushleft}
\end{par}

\begin{matlabcode}
omega = 3;
\end{matlabcode}

\begin{par}
\begin{flushleft}
Se define la variable de la función de transferencia \textit{\textbf{s}}
\end{flushleft}
\end{par}

\begin{matlabcode}
s = tf("s");
\end{matlabcode}

\begin{par}
\begin{flushleft}
Ecuación general para un sistema de segundo orden en el plano \textit{\textbf{s}}\textit{ - }Bolton p129
\end{flushleft}
\end{par}

\begin{matlabcode}
segundo_orden = omega^2 / (s^2 + 2*omega*zeta*s + omega^2)
\end{matlabcode}
\begin{matlaboutput}
segundo_orden =
 
          9
  -----------------
  s^2 + 3.102 s + 9
 
Continuous-time transfer function.
\end{matlaboutput}

\label{H_BD3E9BED}
\matlabheadingthree{Respuesta escalón}

\begin{par}
\hfill \break
\end{par}

\begin{matlabcode}
figure;
step(segundo_orden)
\end{matlabcode}
\begin{center}
\includegraphics[width=\maxwidth{91.82137481184145em}]{figure_0.png}
\end{center}

\label{H_F457F21A}
\matlabheadingthree{Conversión a Polos y Ceros}

\begin{par}
\hfill \break
\end{par}

\begin{matlabcode}
[polos, ceros] = pzmap(segundo_orden);
polos
\end{matlabcode}
\begin{matlaboutput}
polos = 2x1 complex    
  -1.5508 + 2.5681i
  -1.5508 - 2.5681i

\end{matlaboutput}
\begin{matlabcode}
ceros
\end{matlabcode}
\begin{matlaboutput}
ceros =

  0x1 empty double column vector
\end{matlaboutput}


\label{H_B8B726C4}
\matlabheadingtwo{Medidas de Rendimiento}

\label{H_9A442566}
\matlabheadingthree{Valor en Estado estable}

\begin{par}
\hfill \break
\end{par}

\begin{matlabcode}
figure;
[y, t] = step(segundo_orden);
step(segundo_orden) % No encontré otra forma de ocultar salida y hacer plot
title("Valor en estado estable = " + num2str(round(y(end)), 2));
\end{matlabcode}
\begin{center}
\includegraphics[width=\maxwidth{91.82137481184145em}]{figure_1.png}
\end{center}

\label{H_7395B303}
\matlabheadingthree{Delay Time}

\begin{par}
\begin{flushleft}
Tiempo hasta llegar al 50\% del valor estable
\end{flushleft}
\end{par}

\begin{matlabcode}
figure;
tiempo_delay =  (1 + 0.7 * zeta) / omega; % Aproximación
step(segundo_orden)
xlim([tiempo_delay *0.90 tiempo_delay*1.10]); % Tiempo ± 10%
xline(tiempo_delay,'--r');
title("Tiempo hasta llegar al 50% = " + num2str(round(tiempo_delay, 2)))
\end{matlabcode}
\begin{center}
\includegraphics[width=\maxwidth{91.82137481184145em}]{figure_2.png}
\end{center}

\label{H_C00A963F}
\matlabheadingthree{Frecuencia Amortiguada y Ángulo del polo}

\begin{par}
\begin{flushleft}
Varias medidas se expresan en función de la frecuencia amortiguada en vez de la frecuencia natural o utilizan el argumento del polo complejo o ángulo al eje real
\end{flushleft}
\end{par}

\begin{matlabcode}
frecuencia_amortiguada = omega * sqrt(1 - zeta^2)
\end{matlabcode}
\begin{matlaboutput}
frecuencia_amortiguada = 2.5681
\end{matlaboutput}
\begin{matlabcode}
angulo_polo = acos(zeta)
\end{matlabcode}
\begin{matlaboutput}
angulo_polo = 1.0275
\end{matlaboutput}

\label{H_2F628BBE}
\matlabheadingthree{Tiempo de Levantamiento}

\begin{par}
\begin{flushleft}
Tiempo hasta pasar por el valor en estado estable
\end{flushleft}
\end{par}

\begin{matlabcode}
tiempo_levantamiento = (pi - angulo_polo) / frecuencia_amortiguada;

figure;
step(segundo_orden)
xlim([tiempo_levantamiento*0.90 tiempo_levantamiento*1.10]) % Tiempo ± 10%
xline(tiempo_levantamiento,'--r');
title("Tiempo de Levantamiento = " + num2str(round(tiempo_levantamiento, 2)))
\end{matlabcode}
\begin{center}
\includegraphics[width=\maxwidth{91.82137481184145em}]{figure_3.png}
\end{center}

\label{H_95A81755}
\matlabheadingthree{Tiempo del pico}

\begin{par}
\begin{flushleft}
Tiempo hasta alcanzar el valor máximo o el valor de overshoot - Bolton p99
\end{flushleft}
\end{par}

\begin{matlabcode}
tiempo_pico = pi / frecuencia_amortiguada;
procentaje_overshoot = exp(-zeta * pi / sqrt(1 - zeta^2));

figure;
step(segundo_orden)
xlim([tiempo_pico*0.90 tiempo_pico*1.10]) % Tiempo ± 10%
xline(tiempo_pico,'--r');
title("Tiempo del pico = " + num2str(round(tiempo_pico, 2)))
\end{matlabcode}
\begin{center}
\includegraphics[width=\maxwidth{91.82137481184145em}]{figure_4.png}
\end{center}

\label{H_51F24389}
\matlabheadingthree{Tiempo del segundo pico}

\begin{par}
\hfill \break
\end{par}

\begin{matlabcode}
tiempo_segundo_pico = 2*pi / frecuencia_amortiguada;

figure;
step(segundo_orden)
xline(tiempo_segundo_pico,'--r');
title("Tiempo del segundo pico = " + num2str(round(tiempo_segundo_pico, 2)))
\end{matlabcode}
\begin{center}
\includegraphics[width=\maxwidth{91.82137481184145em}]{figure_5.png}
\end{center}

\label{H_4B73A72B}
\matlabheadingthree{Tiempo de Asentamiento}

\begin{par}
\begin{flushleft}
Al 5\%
\end{flushleft}
\end{par}

\begin{matlabcode}
porcentaje_acentamiento = 5;

tiempo_asentamiento = -log(porcentaje_acentamiento/100) / (zeta * omega);

figure;
step(segundo_orden)
xline(tiempo_asentamiento,'--r');
title("Tiempo de Asentamiento al 5% = " + num2str(round(tiempo_asentamiento, 2)))
\end{matlabcode}
\begin{center}
\includegraphics[width=\maxwidth{91.82137481184145em}]{figure_6.png}
\end{center}

\begin{par}
\begin{flushleft}
Al 2\%
\end{flushleft}
\end{par}

\begin{matlabcode}
porcentaje_acentamiento = 2;

tiempo_asentamiento = -log(porcentaje_acentamiento/100) / (zeta * omega);

figure;
step(segundo_orden)
xline(tiempo_asentamiento,'--r');
title("Tiempo de Asentamiento al 2% = " + num2str(round(tiempo_asentamiento, 2)))
\end{matlabcode}
\begin{center}
\includegraphics[width=\maxwidth{91.82137481184145em}]{figure_7.png}
\end{center}

\label{H_4BE83A67}
\matlabheadingthree{Consideraciones}

\begin{par}
\begin{flushleft}
La frecuencia natural \textbf{no afecta el overshoot}
\end{flushleft}
\end{par}

\begin{matlabcode}
figure;

omega_values = [0.05, 0.10, 0.25, 0.50, 1.00];
for index = 1:length(omega_values)
    omega_ = omega_values(index);
    segundo_orden = (omega_^2) / (s^2 + 2*omega_*zeta*s + omega_^2);
    step(segundo_orden)
    legendInfo{index} = ['omega = ' num2str(omega_)]; 
    hold all
end

legend(legendInfo)
frecuencia_amortiguada = 0.05 * sqrt(1 - zeta^2); % Omega mínimo
tiempo_pico = pi / frecuencia_amortiguada;
xlim([0 tiempo_pico*1.1])

overshoot_magnitud = 1 * (1 + PO/100); % Asumiendo step = 1
ylim([overshoot_magnitud*0.95 overshoot_magnitud*1.05]) % Overshoot ± 5%

title("Overshoot para diferentes valores de Omega")
\end{matlabcode}
\begin{center}
\includegraphics[width=\maxwidth{91.82137481184145em}]{figure_8.png}
\end{center}


\label{H_39272037}
\matlabheadingtwo{Polos y Zeros}

\begin{par}
\begin{flushleft}
Para el sistema original
\end{flushleft}
\end{par}

\begin{matlabcode}
segundo_orden = omega^2 / (s^2 + 2*omega*zeta*s + omega^2);

figure;
pzmap(segundo_orden)
grid on
\end{matlabcode}
\begin{center}
\includegraphics[width=\maxwidth{91.82137481184145em}]{figure_9.png}
\end{center}

\begin{par}
\begin{flushleft}
En el caso de tener distintos valores de la frecuencia natural, los polos se ubican sobre la recta que está a cos(zeta) grados del eje real
\end{flushleft}
\end{par}

\begin{matlabcode}
figure;
omega_values = [0.05, 0.10, 0.25, 0.50, 1.00];
for index = 1:length(omega_values)
    omega_ = omega_values(index);
    segundo_orden = (omega_^2) / (s^2 + 2*omega_*zeta*s + omega_^2);
    pzmap(segundo_orden)
    legendInfo{index} = ['omega = ' num2str(omega_)]; 
    hold all
end
legend(legendInfo, 'AutoUpdate', 'off')

% Rectas 

m = tan(angulo_polo); 
x = linspace(-1, 0);
plot(x, m*x, '--', "Color", [0 0 0])
plot(x, -m*x, '--', "Color", [0 0 0])
grid on
\end{matlabcode}
\begin{center}
\includegraphics[width=\maxwidth{91.82137481184145em}]{figure_10.png}
\end{center}

\end{document}
